In [ ]:
# Import necessary libraries
import cv2
import matplotlib.pyplot as plt
from IPython.display import clear_output

# Initialize the HOG descriptor/person detector
hog = cv2.HOGDescriptor()
hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())

# Open video file
video_path = './drive.mp4'  # Replace with the path to your video file
cap = cv2.VideoCapture(video_path)

try:
    while True:
        # Read a new frame
        ret, frame = cap.read()
        if not ret:
            break

        # Detect people in the image
        # Adjust parameters as needed
        boxes, weights = hog.detectMultiScale(frame, winStride=(8, 8), padding=(8, 8), scale=1.05)

        # Draw the detected boxes
        for (x, y, w, h) in boxes:
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

        # Display the resulting frame
        plt.figure(figsize=(10, 10))
        plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        plt.show()
        clear_output(wait=True)

except KeyboardInterrupt:
    print('Interrupted')

cap.release()


In [ ]:
%pip install matplotlib


In [ ]:
import cv2
import torch
import numpy as np
import matplotlib.pyplot as plt

# Load the YOLO model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True).to('mps')

In [ ]:
# Load the YOLO model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True).to('mps')  # Use 'mps' for Apple M1 GPU


In [ ]:
# Load your video
cap = cv2.VideoCapture('./drive.mp4')


In [ ]:
# Prepare for histogram data (example for humans)
human_confidences = []

In [ ]:
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convert frame to RGB (YOLOv5 expects RGB)
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Inference
    results = model([frame_rgb])

    # Extract results
    detections = results.xyxy[0].cpu().numpy()  # Convert detections to NumPy array

    for *xyxy, conf, cls in detections:
        if cls == 0:  # Assuming class 0 is person
            human_confidences.append(conf)
            # For demonstration, let's print the confidence level of each detected human
            print(f'Human detected with confidence: {conf}')

In [ ]:
# Plot histogram of human detection confidences
plt.hist(human_confidences, bins=20, alpha=0.75)
plt.title('Histogram of Human Detection Confidences')
plt.xlabel('Confidence')
plt.ylabel('Frequency')
plt.show()


In [ ]:
plt.hist(human_confidences, bins=20, alpha=0.75)
plt.show()


In [ ]:
cap.release()
cv2.destroyAllWindows()

In [ ]:
%pip install yolov5

In [3]:
import cv2
import torch

# Load the YOLOv5 model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', device='cpu')  # Change 'cpu' to 'mps' for M1

def process_frame(frame):
    # Convert frame from BGR to RGB
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    # Inference
    results = model(frame_rgb)
    # Render results on frame
    frame_result = results.render()[0]
    # Convert frame from RGB back to BGR
    frame_bgr = cv2.cvtColor(frame_result, cv2.COLOR_RGB2BGR)
    return frame_bgr

# Video processing (simplified for demonstration)
cap = cv2.VideoCapture('./drive.mp4')
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    # Process and display frame
    processed_frame = process_frame(frame)
    cv2.imshow('Processed Frame', processed_frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


Using cache found in /Users/abdullahghani/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-3-18 Python-3.12.0 torch-2.2.1 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


In [1]:
import torch
from pathlib import Path
import cv2
import numpy as np

# Load the model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

# Define the video path and output path
video_path = './drive.mp4'
output_path = './output_video.mp4'

# Load the video
cap = cv2.VideoCapture(video_path)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convert frame to RGB (YOLOv5 expects RGB images)
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = model([frame_rgb], size=640)

    # Extract detected humans and their coordinates
    for *xyxy, conf, cls in results.xyxy[0]:
        if int(cls) == 0:  # Class 0 usually corresponds to person in COCO dataset
            label = f'{results.names[int(cls)]} {conf:.2f}'
            cv2.rectangle(frame, (int(xyxy[0]), int(xyxy[1])), (int(xyxy[2]), int(xyxy[3])), (255,0,0), 2)
            cv2.putText(frame, label, (int(xyxy[0]), int(xyxy[1])-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 2)

    out.write(frame)

cap.release()
out.release()


Using cache found in /Users/abdullahghani/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-3-18 Python-3.12.0 torch-2.2.1 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


In [1]:
import torch
import cv2
import numpy as np

def cartoon_effect(frame):
    """Apply a cartoon effect to an image frame."""
    # Convert image to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # Apply median blur
    gray = cv2.medianBlur(gray, 5)
    # Detect edges in the image
    edges = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 9, 9)
    # Convert back to color
    color = cv2.bilateralFilter(frame, 9, 300, 300)
    # Combine edges and color
    cartoon = cv2.bitwise_and(color, color, mask=edges)
    return cartoon

def enhance_effect(frame):
    """Enhance the video frame."""
    # Convert to HSV (hue, saturation, value) color space for easier adjustments
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv)
    # Adjust the value channel to enhance brightness
    v = cv2.equalizeHist(v)
    enhanced_hsv = cv2.merge((h, s, v))
    # Convert back to BGR color space
    enhanced_frame = cv2.cvtColor(enhanced_hsv, cv2.COLOR_HSV2BGR)
    return enhanced_frame

# Load the model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

# Define the video path and output paths
video_path = './drive.mp4'
output_path_cartoon = './output_video_cartoon.mp4'
output_path_enhanced = './output_video_enhanced.mp4'

# Load the video
cap = cv2.VideoCapture(video_path)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Define the codec and create VideoWriter objects
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out_cartoon = cv2.VideoWriter(output_path_cartoon, fourcc, fps, (width, height))
out_enhanced = cv2.VideoWriter(output_path_enhanced, fourcc, fps, (width, height))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Process the frame
    cartoon_frame = cartoon_effect(frame)
    enhanced_frame = enhance_effect(frame)

    # Write the processed frames to the respective videos
    out_cartoon.write(cartoon_frame)
    out_enhanced.write(enhanced_frame)

cap.release()
out_cartoon.release()
out_enhanced.release()


Using cache found in /Users/abdullahghani/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-3-18 Python-3.12.0 torch-2.2.1 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 
